<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/Fraud/Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
db = pd.read_csv('/content/drive/MyDrive/AIEngineer/estimator-med-cost/ราคากลางของบิลแต่ละประเภทในแต่ละโรค.csv')
db = db[['simb_billing_code','local_billing_name','PP','EC','OT','CM','CP','mean']]

In [4]:
db

,simb_billing_code,local_billing_name,PP,EC,OT,CM,CP,mean
0,1.2.3(2),Dental Fees (Other) ค่าทันตแพทย์จากเหตุอื่นๆ,K05,-,-,-,-,850.0
1,1.2.3(2),Dental Fees (Other) ค่าทันตแพทย์จากเหตุอื่นๆ,K050,-,-,-,-,2250.0
2,1.1.5,Diagnostic Radiology/Radiotherapy ค่าตรวจวินิ...,S630,W010,-,-,-,1450.0
3,1.1.5,Diagnostic Radiology/Radiotherapy ค่าตรวจวินิ...,S707,W000,-,-,-,2910.0
4,1.1.5,Diagnostic Radiology/Radiotherapy ค่าตรวจวินิ...,S80.1,W140,-,-,-,1460.0
...,...,...,...,...,...,...,...,...
20568,2.6(2),เบ็ดเตล็ด,S6300,W550,-,-,-,0.0
20569,2.6(2),เบ็ดเตล็ด,S708,W540,-,-,-,0.0
20570,2.6(2),เบ็ดเตล็ด,S8080,W0100,-,-,-,0.0
20571,2.6(2),เบ็ดเตล็ด,S907,W1999,-,-,-,0.0


In [7]:
test = pd.read_csv('/content/drive/MyDrive/AIEngineer/estimator-med-cost/patient.csv')

In [8]:
test.columns

Index(['cid', 'illnes_type', 'chief_complaint', 'total_bill_amount',
       'order_item', 'billing', 'diagnosis', 'icd10', 'dx_name', 'heart_rate',
       'pain_score', 'systolic_bp', 'diastolic_bp', 'temperature',
       'respiratory_rate', 'oxygen_saturation', 'vital_sign_entry_datetime',
       'Severity Index'],
      dtype='object')

In [14]:
df = test.copy()

In [15]:
import ast
# Step 1: Convert diagnosis column to Python objects
df['icd10'] = df['diagnosis'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Step 2: Function to process and split rows for multiple dx_types
def split_diagnoses(row):
    diagnoses = row['icd10']
    dx_dict = {}
    rows = []

    # Group diagnoses by dx_type
    for diag in diagnoses:
        dx_type = diag['dx_type']
        if dx_type not in dx_dict:
            dx_dict[dx_type] = []
        dx_dict[dx_type].append(diag['icd10'])

    # Create rows for each combination of dx_type
    from itertools import product
    keys, values = zip(*dx_dict.items())
    for combination in product(*values):  # Cartesian product of all dx_type combinations
        new_row = row.copy()
        for i, key in enumerate(keys):
            new_row[key] = combination[i]
        rows.append(new_row)
    return rows

# Step 3: Apply the splitting function
processed_rows = []
for _, row in df.iterrows():
    processed_rows.extend(split_diagnoses(row))

final_df = pd.DataFrame(processed_rows)

In [22]:
final_df.loc[[11279]]['diagnosis'].iloc[0]

"[{'icd10': 'K05', 'dx_name': 'เหงือกอักเสบและโรคปริทันต์', 'dx_type': 'PP'}]"

In [5]:
!pip -q install gradio pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00


In [23]:
patient_data = {
    "Patient 1": [{'icd10': 'S711', 'dx_name': 'Open wound of thigh', 'dx_type': 'PP'}, {'icd10': 'W220', 'dx_name': 'Striking against or struck by other object(s)\xa0: at home', 'dx_type': 'EC'}],
    "Patient 2": [{'icd10': 'H102', 'dx_name': 'Other acute conjunctivitis', 'dx_type': 'PP'}],
    "Patient 3": [{"icd10": "S500", "dx_name": "Contusion of elbow", "dx_type": "PP"}, {"icd10": "W200", "dx_name": "Struck by thrown, projected or falling object", "dx_type": "EC"}, {"icd10": "S300", "dx_name": "Contusion of lower back and pelvis", "dx_type": "OT"}, {"icd10": "S400", "dx_name": "Contusion of shoulder and upper arm", "dx_type": "OT"}],
    "Patient 4": [{'icd10': 'K051', 'dx_name': 'Chronic gingivitis', 'dx_type': 'PP'}, {'icd10': 'K028', 'dx_name': 'Other dental caries', 'dx_type': 'OT'}],
    "Patient 5": [{'icd10': 'K05', 'dx_name': 'เหงือกอักเสบและโรคปริทันต์', 'dx_type': 'PP'}],
}

In [6]:
import pandas as pd
import gradio as gr

In [43]:
# Medical cost search functions (from your provided code)
def search_medical_cost(input_json):
    diagnosis_list = eval(input_json)
    query = create_query(diagnosis_list)
    search_results = []

    for _, row in db.iterrows():
        match = True
        for dx_type, icd10_codes in query.items():
            if icd10_codes == "-":  # No codes to match
                if row[dx_type] != "-":
                    match = False
                    break
            else:
                # Ensure an exact match for any of the codes
                if not any(row[dx_type] == code for code in icd10_codes):
                    match = False
                    break
        if match:
            search_results.append(row)

    return pd.DataFrame(search_results)

def create_query(real_icd10):
    query = {"PP": "-", 'EC': '-', 'OT': '-', 'CM': '-', 'CP': '-'}
    for item in real_icd10:
        dx_type = item['dx_type']
        icd10_code = item['icd10']
        if dx_type in query:
            if query[dx_type] == "-":
                query[dx_type] = [icd10_code]
            elif isinstance(query[dx_type], list):
                if icd10_code not in query[dx_type]:
                    query[dx_type].append(icd10_code)
            else:
                query[dx_type] = [query[dx_type], icd10_code]
    return query

In [44]:
def update_input_json(patient_name):
    return gr.update(value=str(patient_data[patient_name]))

def process_input(input_json):
    search_results_df = search_medical_cost(input_json)
    return search_results_df, "Review the table and select rows for summarization."


In [46]:
# Initialize the total sum
selected_sum = 0

# Function to calculate the total from selected rows and manual inputs
def calculate_sum(selected_rows, manual_costs):
    total = 0
    if selected_rows is not None:
        total += sum(row["mean"] for row in selected_rows)
    if manual_costs:
        total += sum(float(cost) for cost in manual_costs.split("+") if cost.strip().isdigit())
    return f"Total Cost: {total:.2f} THB"


In [ ]:
# Main Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### Medical Cost Estimation Demo 📟")

    input_json = gr.Textbox(
        label="Enter Diagnosis JSON",
        placeholder='[{"icd10": "S500", "dx_name": "Contusion of elbow", "dx_type": "PP"}, ...]',
        lines=5,
    )

    # Row of buttons for patients
    with gr.Row():
        for patient_name in patient_data.keys():
            gr.Button(patient_name).click(
                update_input_json, inputs=[gr.State(patient_name)], outputs=[input_json]
            )

    search_button = gr.Button("Search")
    output_table = gr.DataFrame(
        headers=["simb_billing_code", "local_billing_name", "PP", "EC", "OT", "CM", "CP", "mean"],
        interactive=True,
    )
    summary_text = gr.Textbox(label="Summary", interactive=False)

    # Manual cost input
    manual_input = gr.Textbox(
        label="Manual Costs",
        placeholder="e.g., 500, 300, 200",
    )

    # Button to calculate the total
    calculate_button = gr.Button("Calculate Total")

    # Search function to populate the table
    def process_input(input_json):
        search_results_df = search_medical_cost(input_json)
        return search_results_df, "Review the table and select rows for summarization."

    # Link the buttons
    search_button.click(process_input, inputs=[input_json], outputs=[output_table, summary_text])
    calculate_button.click(calculate_sum, inputs=[output_table, manual_input], outputs=[summary_text])

demo.launch(debug=True)

In [61]:
import gradio as gr
import pandas as pd

# Sample DataFrame
sample_data = [
    {"simb_billing_code": "1.2.3(2)", "local_billing_name": "Dental Fees (Other)", "PP": "K05", "EC": "-", "OT": "-", "CM": "-", "CP": "-", "mean": 850},
    {"simb_billing_code": "1.1.12", "local_billing_name": "Nursing and Midwifery Charge", "PP": "K05", "EC": "-", "OT": "-", "CM": "-", "CP": "-", "mean": 155},
    {"simb_billing_code": "1.1.14(2)", "local_billing_name": "Other Medical Service Charge", "PP": "K05", "EC": "-", "OT": "-", "CM": "-", "CP": "-", "mean": 125},
    {"simb_billing_code": "1.1.9(1)", "local_billing_name": "ค่าทันตกรรมเนื่องจากอุบัติเหตุ (AIA)", "PP": "K05", "EC": "-", "OT": "-", "CM": "-", "CP": "-", "mean": 150},
    {"simb_billing_code": "1.1.9(2)", "local_billing_name": "ตรวจฟันและขูดหินปูน (AIA)", "PP": "K05", "EC": "-", "OT": "-", "CM": "-", "CP": "-", "mean": 850},
]
df = pd.DataFrame(sample_data)

# Callback function for row selection and summing
def compute_selected_means(selected_indices):
    if not selected_indices:
        return "No rows selected."
    selected_means = df.iloc[selected_indices]["mean"].tolist()
    selected_str = " + ".join(map(str, selected_means))
    total_sum = sum(selected_means)
    return f"{selected_str} = {total_sum:.2f} THB"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### Select Rows to Sum Their Mean Values")

    # Create a dropdown for row selection
    row_selector = gr.Dropdown(
        choices=[f"Row {i}: {row['local_billing_name']}" for i, row in df.iterrows()],
        label="Select Rows",
        multiselect=True,
    )

    # Display the total sum of selected rows' means
    output_text = gr.Textbox(label="Total Mean of Selected Rows")

    # Link the dropdown to the callback function
    row_selector.change(
        fn=lambda selected: compute_selected_means([int(i.split()[1]) for i in selected] if selected else []),
        inputs=row_selector,
        outputs=output_text,
    )

demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c9e406df905ba83ef1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2045, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1592, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://c9e406df905ba83ef1.gradio.live
